# Stablecoin Simulation

---

This notebook is part of a stablecoin project.

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


---

# 1. System Requirements

## Requirements Analysis

**Goal:** Simulate a simple collateral-backed stablecoin and examine the effects of external price shocks

**Scope:** Showcasing how simulations can offer insights into specific dynamics

**Question:** How do price shocks affect vault liquidations?

**Assumptions:**
- The price of the underlying collateral asset follows a simple stochastic process
- The _Vault Owners_ maintain their vault positions following a certain strategy
- Liquidations are triggered immediately and the collateral flows into the so called _Residual Ecosystem_

## Visual System Mappings

#### Entity Relationship Diagram

TODO

#### Stock & Flow Diagram

TODO

## Mathematical Specification

TODO

---

# 2. System Design

## Differential Specification

TODO

## Modelling

In [ ]:
##############
# 0. IMPORTS #
##############

# Standard libraries
import numpy as np
import copy

# model
from model.state_variables import set_initial_state

# Analysis and plotting modules
import pandas as pd
import plotly.express as px

# radCAD modules
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend


In [ ]:
#############
# 0b. UTILS #
#############

INITIAL_COLL_PRICE = 3000
NUM_OWNERS = 100
MIN_COLL_RATIO = 1.5
LIQUIDATION_RATIO = 1/MIN_COLL_RATIO

# RISKY, RISK-AVERSE, PASSIVE, IRRATIONAL
STRATEGY_DISTRIBUTION = [0.0, 0.0, 0.0, 1.0]


In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = set_initial_state(INITIAL_COLL_PRICE, NUM_OWNERS, LIQUIDATION_RATIO, STRATEGY_DISTRIBUTION)
initial_state

########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    "initial_coll_price": [INITIAL_COLL_PRICE],
    "num_owners": [NUM_OWNERS],
    "liquidation_ratio": [LIQUIDATION_RATIO],
    "stability_fee": [0.05],
    
    # Price simulation parameters, values are defined per year
    "coll_price_drift": [0.2],
    "coll_price_vol": [0.3],
    "jump_rate": [0.25],
    "jump_param_a": [0.1],
    "jump_param_b": [0.2],
}

In [ ]:
# This section is just to examine the jump diffusion price process

from utils.price_simulation import jump_diffusion
import matplotlib.pyplot as plt
import seaborn as sns

num_paths = 10
timesteps = 365
price_params = {
    "coll_price_drift": 0.2,
    "coll_price_vol": 0.3,
    "jump_rate": 0.25,
    "jump_param_a": 0.1,
    "jump_param_b": 0.2,
}
simulated_price_paths = jump_diffusion(100, price_params, num_paths=num_paths, timesteps=timesteps)
simulated_price_changes = np.diff(simulated_price_paths) / simulated_price_paths[:, 1:]

# Choose palette, figure size, and define figure axes
sns.set_theme(palette='viridis')
plt.figure()
width = 15
height = 10
plt.rcParams['figure.figsize'] = [width, height]
fig, ax = plt.subplots(2,1)
fig.tight_layout(pad=5.0)

t = np.linspace(0, timesteps, timesteps+1)

price_path = ax[0].plot(t, simulated_price_paths.transpose());
price_changes = ax[1].plot(t[1:], simulated_price_changes.transpose());

# Make drawn paths thinner by decreasing line width
plt.setp(price_path, linewidth=1);

# Set title (LaTeX notation) and x- and y- labels
ax[0].set(title="Monte Carlo simulated stock price paths in Merton's jump diffusion model\n$S_0$ = {}, $\mu$ = {}, $\sigma$ = {}, $a$ = {}, $b$ = {}, $\lambda$ = {}, timesteps = {}, num_paths = {}".format(100, price_params["coll_price_drift"], price_params["coll_price_vol"], price_params["jump_param_a"], price_params["jump_param_b"], price_params["jump_rate"], timesteps, num_paths), xlabel='Time (days)', ylabel='Stock price')
ax[1].set(title="Price changes in percent")

plt.show()

In [ ]:

##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

from model.policy_functions import p_coll_price, p_liquidation, p_vault_management
from model.state_update_functions import s_collateral, s_owners, s_stability_pool


state_update_blocks = [
    {
        'policies': {
            'coll_price': p_coll_price
        },
        'variables': {
            'collateral': s_collateral,
        }
    },
    {
        'policies': {'liquidation': p_liquidation},
        'variables': {
            'stability_pool': s_stability_pool,
            'owners': s_owners
        }
    },
        {
        'policies': {'vault_management': p_vault_management},
        'variables': {
            'stability_pool': s_stability_pool,
            'owners': s_owners
        }
    }
]

## Simulation

In [ ]:
####################
# 6. CONFIGURATION #
####################

model = Model(
    initial_state=initial_state,
    state_update_blocks=state_update_blocks,
    params=system_params
)

simulation = Simulation(
    model=model,
    timesteps=300,  # Number of timesteps
    runs=10  # Number of Monte Carlo Runs
)

experiment = Experiment([simulation])
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS, drop_substeps=True)


################
# 7. EXECUTION #
################

raw_result = experiment.run()


In [ ]:
####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)

simulation_result['coll_price'] = [t.price for t in simulation_result['collateral']]
simulation_result['coll_price_change'] = simulation_result['coll_price'].pct_change()
simulation_result['stability_pool_balance'] = [t.stable_coin_balance for t in simulation_result['stability_pool']]
simulation_result['vault_collateral'] = [t[0].vault.collateral_balance for t in simulation_result['owners']]
simulation_result['vault_debt'] = [t[0].vault.debt_balance for t in simulation_result['owners']]
simulation_result['wallet_collateral'] = [t[0].wallet.collateral_balance for t in simulation_result['owners']]
simulation_result['wallet_stable_coin'] = [t[0].wallet.stable_coin_balance for t in simulation_result['owners']]
# simulation_result.drop(columns=['owners', 'collateral', 'stability_pool'], inplace=True)

simulation_result.head(10)

In [ ]:
###################################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE CHANGE #
###################################################

pd.options.plotting.backend = "plotly"
simulation_result[simulation_result["timestep"]!= 0].plot(
    kind='line',
    x='timestep',
    y=['coll_price_change'],
    title="Collateral Price Change Rate",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_price'],
    title="Collateral Price",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: STABILITY POOL #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['stability_pool_balance'],
    title="Stability Pool Balance",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: VAULT #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['vault_debt'],
    title="Vault Debt",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: VAULT #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['vault_collateral'],
    title="Vault Collateral",
    color='run'
)